# 0. Import Module and Information

In [ ]:
import time, re
from selenium import webdriver
from bs4 import BeautifulSoup
import random, os
import pandas as pd

# **입력 필요!!**

In [ ]:
user_id = '인스타그램 아이디 입력'
user_password = '인스타그램 비밀번호 입력'
driver_path = 'chromedriver.exe 경로' # 셀레니움 이용을 위해 필요
tag_word = '서울핫플' # 검색하고 싶은 키워드
save_path = '크롤링 결과값 저장 경로'
wish_num = 100 # 크롤링할 포스트 개수

# 1. Open Browser and Login

## Open Browser

In [ ]:
driver = webdriver.Chrome(driver_path)

url = 'https://www.instagram.com'

driver.get(url)
time.sleep(10)

## Login

In [ ]:
try:
    input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
    input_id.clear()
    input_id.send_keys(user_id)
    
    input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
    input_pw.clear()
    input_pw.send_keys(user_password)
    
    input_pw.submit()
    time.sleep(5)
    
except:
    print("instagram login fail")

## 로그인 정보 저장 창이 나올시 '나중에 하기' 선택

In [ ]:
try:
    driver.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div/div/section/div/button').click()
except:
    pass

# 2. Search Keyword

In [ ]:
def insta_search(word):
    url = 'https://www.instagram.com/explore/tags/' + word
    return url 

url = insta_search(tag_word)
driver.get(url)
time.sleep(40)

# 3. Select First Image

In [ ]:
def select_first(driver):
    first = driver.find_element_by_css_selector('div._aagw')
    first.click()
    time.sleep(5)

select_first(driver)

# **여기 중요!!**

1. 셀레니움이 실행 중인 크롬 창에서 현재 하나의 게시물이 떠있을 것

2. 시간 나오는 부분에서 마우스 우클릭 -> 검사

3. 그러면 개발자도구가 뜨고 강조되는 부분이 있다

4. 그 부분에서 마우스 우클릭 -> Copy -> Copy Selector 클릭

5. 그러면 #mount_0_0_pa > div > div:nth-child(1) > div > div:nth-child(4) > div ....과 같은 값이 복사가 되는데 여기서 필요한 값음 #mount_0_0_pa의 pa

6. 이 값을 아래 mount 변수에 입력

In [ ]:
mount = 'pa' # 여기에 mount_0_0_@@에서 @@ 입력

# 4. Crawling and Next Arrow

In [ ]:
def making_css_selector(mount):
    loc_sel = '#mount_0_0_' + mount + ' > div > div:nth-child(1) > div > div:nth-child(4) > div > div > div.rq0escxv.l9j0dhe7.du4w35lb > div > div.iqfcb0g7.tojvnm2t.a6sixzi8.k5wvi7nf.q3lfd5jv.pk4s997a.bipmatt0.cebpdrjk.qowsmv63.owwhemhu.dp1hu0rb.dhp61c6y.l9j0dhe7.iyyx5f41.a8s20v7p > div > div > div > div > div.pi61vmqs.od1n8kyl.h6an9nv3.j4yusqav.djyw54ux.c9k30104.rxghi256.jhx0qe0y.mbxd2wpa.hb7lyos6.rfyvs5rk.sspdcydq.n34oi56o.c3wyshyw.im6prb7w.kzdz7bm1.k01ojvdi.alxbew3a.t78t6opn > div > article > div > div._aata > div > div > div._aasx > div._aat8 > div > div > a > div > time'
    date_sel = '#mount_0_0_' + mount + ' > div > div:nth-child(1) > div > div:nth-child(4) > div > div > div.rq0escxv.l9j0dhe7.du4w35lb > div > div.iqfcb0g7.tojvnm2t.a6sixzi8.k5wvi7nf.q3lfd5jv.pk4s997a.bipmatt0.cebpdrjk.qowsmv63.owwhemhu.dp1hu0rb.dhp61c6y.l9j0dhe7.iyyx5f41.a8s20v7p > div > div > div > div > div.pi61vmqs.od1n8kyl.h6an9nv3.j4yusqav.djyw54ux.c9k30104.rxghi256.jhx0qe0y.mbxd2wpa.hb7lyos6.rfyvs5rk.sspdcydq.n34oi56o.c3wyshyw.im6prb7w.kzdz7bm1.k01ojvdi.alxbew3a.t78t6opn > div > article > div > div._aata > div > div > div._aasx > div._aat8 > div > div > a > div > time'
    main_sel = '#mount_0_0_' + mount + ' > div > div:nth-child(1) > div > div:nth-child(4) > div > div > div.rq0escxv.l9j0dhe7.du4w35lb > div > div.iqfcb0g7.tojvnm2t.a6sixzi8.k5wvi7nf.q3lfd5jv.pk4s997a.bipmatt0.cebpdrjk.qowsmv63.owwhemhu.dp1hu0rb.dhp61c6y.l9j0dhe7.iyyx5f41.a8s20v7p > div > div > div > div > div.pi61vmqs.od1n8kyl.h6an9nv3.j4yusqav.djyw54ux.c9k30104.rxghi256.jhx0qe0y.mbxd2wpa.hb7lyos6.rfyvs5rk.sspdcydq.n34oi56o.c3wyshyw.im6prb7w.kzdz7bm1.k01ojvdi.alxbew3a.t78t6opn > div > article > div > div._aata > div > div > div._aasx > div._aat6 > ul > div > li > div > div > div._a9zr > div._a9zs'
    next_arrow_sel = '#mount_0_0_' + mount + ' > div > div:nth-child(1) > div > div:nth-child(4) > div > div > div.rq0escxv.l9j0dhe7.du4w35lb > div > div.iqfcb0g7.tojvnm2t.a6sixzi8.k5wvi7nf.q3lfd5jv.pk4s997a.bipmatt0.cebpdrjk.qowsmv63.owwhemhu.dp1hu0rb.dhp61c6y.l9j0dhe7.iyyx5f41.a8s20v7p > div > div > div > div > div._a3gq._ab-1 > div > div > div._aaqg._aaqh > button > div'
    
    return loc_sel, date_sel, main_sel, next_arrow_sel

loc_sel, date_sel, main_sel, next_arrow_sel = making_css_selector(mount)

## 위치 정보

In [ ]:
def get_location_data(driver):
    try:
        location_object = driver.find_element_by_css_selector(loc_sel)
        location_info = location_object.text
        location_href = location_object.get_attribute("href")
    except:
        location_info = None
        location_href = None

    return location_info, location_href

## 날짜 정보

In [ ]:
def get_date_info(driver):
    try:
        date_object = driver.find_element_by_css_selector(date_sel))
        date_time = date_object.get_attribute("datetime")
        date_title = date_object.get_attribute("title")
    except:
        date_time = None
        date_title = None

    return date_time, date_title

## 본문 및 해시태그

In [ ]:
def get_main_text(driver, instagram_tags):
    try:
        main_text_object = driver.find_element_by_css_selector(main_sel)
        main_text = main_text_object.text
    except:
        main_text = None

    try:
        data = driver.find_element_by_css_selector(main_sel)
        tag_raw = data.text
        tags = re.findall('#[A-Za-z0-9가-힣]+', tag_raw)
        tag = ''.join(tags).replace("#", " ")

        tag_data = tag.split()

        for tag_one in tag_data:
            instagram_tags.append(tag_one)
    except:
        pass

    return main_text, instagram_tags

## 다음 버튼 클릭

In [ ]:
def click_next_arrow_button(driver):
    try:
        right= driver.find_element_by_css_selector(next_arrow_sel)
        right.click()
        time.sleep(5.0)
    except:
        pass

## 기타 util 함수

In [ ]:
def save_extract_data_to_csv_file(location_infos, location_hrefs, 
                                  date_times, date_titles, main_texts, save_file_name):
    os.chdir(save_path)
    try:
        insta_info_df = pd.DataFrame(
            {"location_info": location_infos, "location_href": location_hrefs,
             "date_time": date_times, "date_title": date_titles, "main_text": main_texts,})
        insta_info_df.to_csv("{}.csv".format(save_file_name), index=True)
    except:
        print("Fail to save extract data to csv file")


def save_extract_tag_data_to_csv_file(instagram_tags, save_file_name_tag):
    os.chdir(save_path)
    try:
        insta_tag_df = pd.DataFrame({"tag": instagram_tags})
        insta_tag_df.to_csv("{}.csv".format(save_file_name_tag), index=True)
    except:
        print("Fail to save extract tag data to csv file")


def make_radom_sleep_time(start, end):
    return random.randrange(start=start, stop=end+1)

# 5. main function

In [ ]:
location_infos, location_hrefs = [], []
date_times, date_titles = [], []
main_texts, instagram_tags = [], []

save_file_name, save_tag_file_name = "extracted_main", "extracted_tag"


count_extract_num = 0

while True:
    if count_extract_num > wish_num:
        break

    time.sleep(make_radom_sleep_time(start=5, end=9))

    location_info, location_href = get_location_data(driver=driver)

    date_time, date_title = get_date_info(driver=driver)

    main_text, instagram_tags = get_main_text(driver=driver, instagram_tags=instagram_tags)

    count_extract_num += 1

    location_infos.append(location_info)
    location_hrefs.append(location_href)
    date_times.append(date_time)
    date_titles.append(date_title)
    main_texts.append(main_text)

    time.sleep(make_radom_sleep_time(start=5, end=8))

    click_next_arrow_button(driver=driver)

save_extract_data_to_csv_file(location_infos=location_infos, location_hrefs=location_hrefs,
                             date_times=date_times, date_titles=date_titles,
                             main_texts=main_texts, save_file_name=save_file_name)

save_extract_tag_data_to_csv_file(instagram_tags=instagram_tags, save_file_name_tag=save_tag_file_name)

driver.close()